# Experiment 008: Feature Selection + CatBoost

Key learning from exp_006: **CV is king** (R²=0.92 between CV and LB).
To beat exp_003's LB of 0.8045, need CV > 0.8195.

Approach:
1. Remove 22 features with importance < 1.0
2. Retrain CatBoost with best params from exp_003
3. Goal: Reduce overfitting, potentially improve CV

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings('ignore')

# Load data
train = pd.read_csv('/home/data/train.csv')
test = pd.read_csv('/home/data/test.csv')
print(f"Train: {train.shape}, Test: {test.shape}")

Train: (8693, 14), Test: (4277, 13)


In [2]:
# Feature engineering (same as previous experiments)
def feature_engineering(df):
    df = df.copy()
    df['Group'] = df['PassengerId'].apply(lambda x: int(x.split('_')[0]))
    df['PassengerNum'] = df['PassengerId'].apply(lambda x: int(x.split('_')[1]))
    df['Deck'] = df['Cabin'].apply(lambda x: x.split('/')[0] if pd.notna(x) else np.nan)
    df['CabinNum'] = df['Cabin'].apply(lambda x: int(x.split('/')[1]) if pd.notna(x) else np.nan)
    df['Side'] = df['Cabin'].apply(lambda x: x.split('/')[2] if pd.notna(x) else np.nan)
    return df

train = feature_engineering(train)
test = feature_engineering(test)

# GroupSize
all_data = pd.concat([train[['Group']], test[['Group']]], ignore_index=True)
group_sizes = all_data['Group'].value_counts().to_dict()
for df in [train, test]:
    df['GroupSize'] = df['Group'].map(group_sizes)
    df['Solo'] = (df['GroupSize'] == 1).astype(int)
print("Basic features done")

Basic features done


In [3]:
# Group-based imputation
def group_based_imputation(train_df, test_df):
    combined = pd.concat([train_df, test_df], ignore_index=True)
    for col in ['HomePlanet', 'Deck', 'Side']:
        group_mode = combined.groupby('Group')[col].apply(
            lambda x: x.mode()[0] if len(x.mode()) > 0 else np.nan).to_dict()
        for df in [train_df, test_df]:
            mask = df[col].isna()
            df.loc[mask, col] = df.loc[mask, 'Group'].map(group_mode)
    return train_df, test_df

train, test = group_based_imputation(train, test)

# Remaining imputation
def impute_remaining(df):
    df = df.copy()
    spending_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    for col in spending_cols:
        mask = (df['CryoSleep'] == True) & (df[col].isna())
        df.loc[mask, col] = 0
    for col in ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']:
        if df[col].isna().any():
            df[col] = df[col].fillna(df[col].mode()[0])
    for col in ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'CabinNum']:
        if df[col].isna().any():
            df[col] = df[col].fillna(df[col].median())
    return df

train = impute_remaining(train)
test = impute_remaining(test)
print("Imputation done")

Imputation done


In [4]:
# Create features
def create_features(df):
    df = df.copy()
    spending_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    
    df['TotalSpent'] = df[spending_cols].sum(axis=1)
    for col in spending_cols:
        df[f'{col}_ratio'] = df[col] / (df['TotalSpent'] + 1)
        df[f'{col}_spent'] = (df[col] > 0).astype(int)
        df[f'{col}_log'] = np.log1p(df[col])
    
    df['LuxurySpent'] = df['Spa'] + df['VRDeck'] + df['RoomService']
    df['BasicSpent'] = df['FoodCourt'] + df['ShoppingMall']
    df['LuxuryRatio'] = df['LuxurySpent'] / (df['TotalSpent'] + 1)
    df['SpentPerAge'] = df['TotalSpent'] / (df['Age'] + 1)
    df['SpendingBin'] = pd.cut(df['TotalSpent'], bins=[-1, 0, 500, 2000, float('inf')], labels=[0, 1, 2, 3]).astype(int)
    df['NumSpendingCategories'] = sum(df[f'{col}_spent'] for col in spending_cols)
    df['TotalSpent_log'] = np.log1p(df['TotalSpent'])
    df['LuxurySpent_log'] = np.log1p(df['LuxurySpent'])
    df['BasicSpent_log'] = np.log1p(df['BasicSpent'])
    df['Spa_VRDeck_RoomService'] = df['Spa'] + df['VRDeck'] + df['RoomService']
    df['FoodCourt_RoomService'] = df['FoodCourt'] + df['RoomService']
    
    df['CryoSleep_HomePlanet'] = df['CryoSleep'].astype(str) + '_' + df['HomePlanet'].astype(str)
    df['CryoSleep_Destination'] = df['CryoSleep'].astype(str) + '_' + df['Destination'].astype(str)
    df['Deck_Side'] = df['Deck'].astype(str) + '_' + df['Side'].astype(str)
    df['HomePlanet_Destination'] = df['HomePlanet'].astype(str) + '_' + df['Destination'].astype(str)
    df['AgeGroup'] = pd.cut(df['Age'], bins=[0, 12, 17, 25, 40, 60, 100], 
                            labels=['Child', 'Teen', 'YoungAdult', 'Adult', 'MiddleAge', 'Senior']).astype(str)
    df['AgeGroup_CryoSleep'] = df['AgeGroup'] + '_' + df['CryoSleep'].astype(str)
    df['VIP_HomePlanet'] = df['VIP'].astype(str) + '_' + df['HomePlanet'].astype(str)
    
    df['IsChild'] = (df['Age'] <= 12).astype(int)
    df['IsTeen'] = ((df['Age'] > 12) & (df['Age'] <= 17)).astype(int)
    df['IsYoungAdult'] = ((df['Age'] > 17) & (df['Age'] <= 25)).astype(int)
    df['IsAdult'] = ((df['Age'] > 25) & (df['Age'] <= 60)).astype(int)
    df['IsSenior'] = (df['Age'] > 60).astype(int)
    return df

train = create_features(train)
test = create_features(test)
print("Features created")

Features created


In [5]:
# Encode categorical features
cat_features = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side',
                'CryoSleep_HomePlanet', 'CryoSleep_Destination', 'Deck_Side',
                'HomePlanet_Destination', 'AgeGroup', 'AgeGroup_CryoSleep', 'VIP_HomePlanet']

for col in cat_features:
    le = LabelEncoder()
    combined = pd.concat([train[col].astype(str), test[col].astype(str)])
    le.fit(combined)
    train[col + '_enc'] = le.transform(train[col].astype(str))
    test[col + '_enc'] = le.transform(test[col].astype(str))

print("Encoding done")

Encoding done


In [6]:
# Define ALL features (56 total)
all_feature_cols = [
    # Encoded categoricals (13)
    'HomePlanet_enc', 'CryoSleep_enc', 'Destination_enc', 'VIP_enc', 'Deck_enc', 'Side_enc',
    'CryoSleep_HomePlanet_enc', 'CryoSleep_Destination_enc', 'Deck_Side_enc',
    'HomePlanet_Destination_enc', 'AgeGroup_enc', 'AgeGroup_CryoSleep_enc', 'VIP_HomePlanet_enc',
    # Numerical (43)
    'Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
    'Group', 'PassengerNum', 'CabinNum', 'GroupSize', 'Solo',
    'TotalSpent', 'LuxurySpent', 'BasicSpent', 'LuxuryRatio', 'SpentPerAge', 'SpendingBin',
    'NumSpendingCategories', 'Spa_VRDeck_RoomService', 'FoodCourt_RoomService',
    'RoomService_ratio', 'FoodCourt_ratio', 'ShoppingMall_ratio', 'Spa_ratio', 'VRDeck_ratio',
    'RoomService_spent', 'FoodCourt_spent', 'ShoppingMall_spent', 'Spa_spent', 'VRDeck_spent',
    'RoomService_log', 'FoodCourt_log', 'ShoppingMall_log', 'Spa_log', 'VRDeck_log',
    'TotalSpent_log', 'LuxurySpent_log', 'BasicSpent_log',
    'IsChild', 'IsTeen', 'IsYoungAdult', 'IsAdult', 'IsSenior'
]

print(f"Total features: {len(all_feature_cols)}")

Total features: 56


In [7]:
# First, train with ALL features to get feature importances
X_all = train[all_feature_cols].values
y = train['Transported'].astype(int).values
X_test_all = test[all_feature_cols].values

# Best CatBoost params from exp_003
cat_params = {
    'depth': 8,
    'learning_rate': 0.051,
    'iterations': 755,
    'l2_leaf_reg': 3.52,
    'random_seed': 42,
    'verbose': False
}

# Train once to get feature importances
model_full = CatBoostClassifier(**cat_params)
model_full.fit(X_all, y)

# Get feature importances
importances = model_full.feature_importances_
importance_df = pd.DataFrame({
    'feature': all_feature_cols,
    'importance': importances
}).sort_values('importance', ascending=False)

print("Feature importances (top 20):")
print(importance_df.head(20))
print(f"\nFeatures with importance < 1.0: {(importance_df['importance'] < 1.0).sum()}")

Feature importances (top 20):
                       feature  importance
21                    CabinNum   10.029196
8                Deck_Side_enc    8.008531
27                 LuxuryRatio    7.561437
19                       Group    6.537772
13                         Age    4.915260
4                     Deck_enc    4.620475
5                     Side_enc    4.443855
0               HomePlanet_enc    2.780185
28                 SpentPerAge    2.549511
6     CryoSleep_HomePlanet_enc    2.492129
9   HomePlanet_Destination_enc    2.483845
24                  TotalSpent    2.140316
2              Destination_enc    2.085102
49             LuxurySpent_log    2.053456
22                   GroupSize    1.794456
34             FoodCourt_ratio    1.741531
36                   Spa_ratio    1.728501
31      Spa_VRDeck_RoomService    1.719974
35          ShoppingMall_ratio    1.687209
25                 LuxurySpent    1.651162

Features with importance < 1.0: 24


In [8]:
# Features to REMOVE (importance < 1.0)
low_importance_features = importance_df[importance_df['importance'] < 1.0]['feature'].tolist()
print(f"\nRemoving {len(low_importance_features)} low-importance features:")
print(low_importance_features)

# Selected features (importance >= 1.0)
selected_features = importance_df[importance_df['importance'] >= 1.0]['feature'].tolist()
print(f"\nKeeping {len(selected_features)} features")


Removing 24 low-importance features:
['Spa', 'Spa_log', 'FoodCourt_log', 'VRDeck_log', 'AgeGroup_enc', 'ShoppingMall_log', 'RoomService_log', 'FoodCourt', 'VRDeck', 'RoomService', 'IsTeen', 'Solo', 'IsYoungAdult', 'SpendingBin', 'CryoSleep_enc', 'FoodCourt_spent', 'IsAdult', 'Spa_spent', 'VRDeck_spent', 'ShoppingMall_spent', 'IsSenior', 'IsChild', 'RoomService_spent', 'VIP_enc']

Keeping 32 features


In [9]:
# Train CatBoost with SELECTED features only
X_selected = train[selected_features].values
X_test_selected = test[selected_features].values

n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

oof_selected = np.zeros(len(X_selected))
test_selected = np.zeros(len(X_test_selected))
fold_scores_selected = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_selected, y)):
    X_train, X_val = X_selected[train_idx], X_selected[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    model = CatBoostClassifier(**cat_params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=100)
    
    oof_selected[val_idx] = model.predict_proba(X_val)[:, 1]
    test_selected += model.predict_proba(X_test_selected)[:, 1] / n_folds
    
    fold_acc = accuracy_score(y_val, (oof_selected[val_idx] >= 0.5).astype(int))
    fold_scores_selected.append(fold_acc)
    print(f"Fold {fold+1}: {fold_acc:.5f}")

selected_acc = accuracy_score(y, (oof_selected >= 0.5).astype(int))
print(f"\nSelected Features CatBoost CV: {selected_acc:.5f} (+/- {np.std(fold_scores_selected):.5f})")

Fold 1: 0.81311


Fold 2: 0.81024


Fold 3: 0.81714


Fold 4: 0.82048


Fold 5: 0.80667

Selected Features CatBoost CV: 0.81353 (+/- 0.00489)


In [10]:
# Compare with ALL features
oof_all = np.zeros(len(X_all))
test_all = np.zeros(len(X_test_all))
fold_scores_all = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_all, y)):
    X_train, X_val = X_all[train_idx], X_all[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    model = CatBoostClassifier(**cat_params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=100)
    
    oof_all[val_idx] = model.predict_proba(X_val)[:, 1]
    test_all += model.predict_proba(X_test_all)[:, 1] / n_folds
    
    fold_acc = accuracy_score(y_val, (oof_all[val_idx] >= 0.5).astype(int))
    fold_scores_all.append(fold_acc)
    print(f"Fold {fold+1}: {fold_acc:.5f}")

all_acc = accuracy_score(y, (oof_all >= 0.5).astype(int))
print(f"\nAll Features CatBoost CV: {all_acc:.5f} (+/- {np.std(fold_scores_all):.5f})")

Fold 1: 0.82001


Fold 2: 0.81196


Fold 3: 0.82231


Fold 4: 0.81761


Fold 5: 0.80898

All Features CatBoost CV: 0.81617 (+/- 0.00498)


In [11]:
# Comparison
print("\n=== COMPARISON ===")
print(f"All features ({len(all_feature_cols)}):      CV = {all_acc:.5f}")
print(f"Selected features ({len(selected_features)}): CV = {selected_acc:.5f}")
print(f"exp_003 (best LB):           CV = 0.81951")
print(f"\nImprovement from feature selection: {selected_acc - all_acc:+.5f}")

# Choose best model
if selected_acc > all_acc:
    print("\n>>> Using SELECTED features (better CV)")
    final_test = test_selected
    final_cv = selected_acc
else:
    print("\n>>> Using ALL features (better CV)")
    final_test = test_all
    final_cv = all_acc


=== COMPARISON ===
All features (56):      CV = 0.81617
Selected features (32): CV = 0.81353
exp_003 (best LB):           CV = 0.81951

Improvement from feature selection: -0.00265

>>> Using ALL features (better CV)


In [12]:
# Create submission
test_binary = (final_test >= 0.5).astype(bool)
submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Transported': test_binary
})

submission.to_csv('/home/submission/submission.csv', index=False)
print(f"Submission saved")
print(f"Final CV: {final_cv:.5f}")
print(f"Predicted transported rate: {test_binary.mean():.4f}")
print(f"Training transported rate: {y.mean():.4f}")
print(submission.head())

Submission saved
Final CV: 0.81617
Predicted transported rate: 0.5198
Training transported rate: 0.5036
  PassengerId  Transported
0     0013_01         True
1     0018_01        False
2     0019_01         True
3     0021_01         True
4     0023_01         True
